# Heart Statlog dataset with LionForests V2

In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-43]) 
sys.path.append(cpath[0][:-43]+'/algorithms')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular


Firstly, we load the dataset and we set the feature and class names

In [3]:
heart = Dataset()
X, y, feature_names, class_names = heart.load_heart() #Feature names should not contain any space
y = np.array(y)
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

Then we initiate LionForests

In [5]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, iterr):
    parameters = [{
        'max_depth': [5],
        'max_features': [0.75],
        'bootstrap': [False],
        'min_samples_leaf' : [5],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            te = time.time()
            return rule, cover, def_predictions, te-ts

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        from CHIRPS.structures import data_container
        import CHIRPS.routines as rt
        import CHIRPS.structures as strcts
        project_dir = cpath[0][:-43]+'/algorithms/CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'heart_'+str(iterr), random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)
        
        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    
    
    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    
    for test_ind in range(len(test)):
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response

In [6]:
total_results = []
kf = KFold(n_splits=10)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    test_size.append(len(X_test))
    print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names, folds)
    folds=folds+1
    total_results.append(results)

# of Fold: 1, size of test: 27
len self.path_detail
27
Working on CHIRPS for instance 0 of 27
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 43 patterns from 483 trees for batch_idx 0
start score sort for batch_idx 0 (43) patterns
start merge rule for batch_idx 0 (43) patterns
[('chest_pain', False, 0.5416666716337204), ('number_of_major_vessels', False, -0.66667)]
0.9682539682539683 0.24796747967479674 0.15495823092837643 0.261579818031431
merge complete for batch_idx 0 (43) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 46 patterns from 352 trees for batch_idx 1
start score sort for batch_idx 1 (46) patterns
start merge rule for batch_idx 1 (46) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.9578947368421052 0.37063605930176946 0.1449121668960813 0.17859834212509426
merge complete for batch_idx 1 (46) patterns
start get explainer for 

[('chest_pain', False, 0.5416666716337204), ('reversable_defect', False, 0.375)]
0.9295774647887324 0.2745199792423456 0.23088482843089647 0.3322033898305085
merge complete for batch_idx 17 (34) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 51 patterns from 315 trees for batch_idx 18
start score sort for batch_idx 18 (51) patterns
start merge rule for batch_idx 18 (51) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.979381443298969 0.3861788617886179 0.13034579896162912 0.2326293235384146
merge complete for batch_idx 18 (51) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
found 41 patterns from 474 trees for batch_idx 19
start score sort for batch_idx 19 (41) patterns
start merge rule for batch_idx 19 (41) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.957894736

found 41 patterns from 460 trees for batch_idx 8
start score sort for batch_idx 8 (41) patterns
start merge rule for batch_idx 8 (41) patterns
[('chest_pain', False, 0.5416666716337204), ('number_of_major_vessels', False, -0.7916666716337204)]
0.9552238805970149 0.2623668842322226 0.16928820620370777 0.25225820665344784
merge complete for batch_idx 8 (41) patterns
start get explainer for batch_idx 8
as_chirps for batch_idx 9
start mining for batch_idx 9 with support = 0.1
found 31 patterns from 491 trees for batch_idx 9
start score sort for batch_idx 9 (31) patterns
start merge rule for batch_idx 9 (31) patterns
[('chest_pain', False, 0.5416666716337204), ('number_of_major_vessels', False, -0.7916666716337204)]
0.9552238805970149 0.2623668842322226 0.15420993531384186 0.23953539338154725
merge complete for batch_idx 9 (31) patterns
start get explainer for batch_idx 9
Working on CHIRPS for instance 10 of 27
as_chirps for batch_idx 10
start mining for batch_idx 10 with support = 0.1
foun

[('chest_pain', True, 0.66667), ('oldpeak', True, -0.36882), ('reversable_defect', True, -0.25)]
0.9767441860465116 0.34146341463414637 0.14249442477983804 0.12307007125890734
merge complete for batch_idx 25 (59) patterns
start get explainer for batch_idx 25
as_chirps for batch_idx 26
start mining for batch_idx 26 with support = 0.1
found 72 patterns from 494 trees for batch_idx 26
start score sort for batch_idx 26 (72) patterns
start merge rule for batch_idx 26 (72) patterns
[('chest_pain', True, 0.66667), ('oldpeak', True, -0.24436), ('reversable_defect', True, -0.25)]
0.9770114942528736 0.34552845528455284 0.2034699915981834 0.30142156246068696
merge complete for batch_idx 26 (72) patterns
start get explainer for batch_idx 26
# of Fold: 3, size of test: 27
len self.path_detail
27
Working on CHIRPS for instance 0 of 27
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 47 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (47) patter

[('chest_pain', True, 0.66667), ('oldpeak', True, -0.19331), ('reversable_defect', True, -0.25)]
0.9767441860465116 0.34146341463414637 0.15703396252793345 0.24718019257221469
merge complete for batch_idx 15 (59) patterns
start get explainer for batch_idx 15
as_chirps for batch_idx 16
start mining for batch_idx 16 with support = 0.1
reduced 3 patterns out of 58 by numeric redundancy
found 55 patterns from 500 trees for batch_idx 16
start score sort for batch_idx 16 (55) patterns
start merge rule for batch_idx 16 (55) patterns
[('number_of_major_vessels', False, -0.7916666716337204), ('reversable_defect', False, -0.25)]
0.967741935483871 0.24390243902439024 0.09292696436272248 0.16571811181787746
merge complete for batch_idx 16 (55) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 52 patterns from 338 trees for batch_idx 17
start score sort for batch_idx 17 (52) patterns
start merge rule for batch_idx 17 (52)

...

# of Fold: 8, size of test: 27
len self.path_detail
27
Working on CHIRPS for instance 0 of 27
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 46 patterns from 498 trees for batch_idx 0
start score sort for batch_idx 0 (46) patterns
start merge rule for batch_idx 0 (46) patterns
[('number_of_major_vessels', False, -0.7916666716337204), ('reversable_defect', False, -0.25)]
0.9354838709677419 0.2404428299602145 0.13515046193515812 0.21749835850295476
merge complete for batch_idx 0 (46) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 70 patterns from 481 trees for batch_idx 1
start score sort for batch_idx 1 (70) patterns
start merge rule for batch_idx 1 (70) patterns
[('number_of_major_vessels', True, -0.66667), ('reversable_defect', True, -0.25), ('oldpeak', True, -0.32258)]
0.9583333333333334 0.3746213932727563 0.20386767006008766 0.28587428251434954
merge complete for batch_idx 

[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25), ('oldpeak', True, -0.45647)]
0.9767441860465116 0.34146341463414637 0.14937370969495223 0.1951332026294545
merge complete for batch_idx 17 (68) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 55 patterns from 404 trees for batch_idx 18
start score sort for batch_idx 18 (55) patterns
start merge rule for batch_idx 18 (55) patterns
[('chest_pain', True, 0.66667), ('number_of_major_vessels', True, -0.66667)]
0.9333333333333333 0.34369520165789896 0.08304227468168357 0.11103339744710614
merge complete for batch_idx 18 (55) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
found 32 patterns from 261 trees for batch_idx 19
start score sort for batch_idx 19 (32) patterns
start merge rule for batch_idx 19 (32) patterns
[('exercise_induced_angina', False, -0.125), ('reversable_d

[('number_of_major_vessels', False, -0.7916666716337204), ('reversable_defect', False, -0.25)]
0.9666666666666667 0.23577235772357724 0.137052744878849 0.21985976023524084
merge complete for batch_idx 7 (39) patterns
start get explainer for batch_idx 7
as_chirps for batch_idx 8
start mining for batch_idx 8 with support = 0.1
found 52 patterns from 500 trees for batch_idx 8
start score sort for batch_idx 8 (52) patterns
start merge rule for batch_idx 8 (52) patterns
[('number_of_major_vessels', True, -0.66667), ('resting_blood_pressure', True, -0.03748), ('reversable_defect', True, -0.25)]
0.9642857142857143 0.33015873015873015 0.4747932467042318 0.5740940632228217
merge complete for batch_idx 8 (52) patterns
start get explainer for batch_idx 8
as_chirps for batch_idx 9
start mining for batch_idx 9 with support = 0.1
reduced 8 patterns out of 88 by numeric redundancy
found 80 patterns from 371 trees for batch_idx 9
start score sort for batch_idx 9 (80) patterns
start merge rule for batc

[('exercise_induced_angina', False, -0.125), ('reversable_defect', False, -0.25)]
0.8703703703703703 0.2037233415812419 0.740961904495244 0.8584384858044164
merge complete for batch_idx 24 (23) patterns
start get explainer for batch_idx 24
Working on CHIRPS for instance 25 of 27
as_chirps for batch_idx 25
start mining for batch_idx 25 with support = 0.1
found 54 patterns from 493 trees for batch_idx 25
start score sort for batch_idx 25 (54) patterns
start merge rule for batch_idx 25 (54) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.9484536082474226 0.3751451800232288 0.08718445466320596 0.16402043352202128
merge complete for batch_idx 25 (54) patterns
start get explainer for batch_idx 25
as_chirps for batch_idx 26
start mining for batch_idx 26 with support = 0.1
found 53 patterns from 489 trees for batch_idx 26
start score sort for batch_idx 26 (53) patterns
start merge rule for batch_idx 26 (53) patterns
[('number_of_major_vessels', True, -0.66667), (

[('chest_pain', False, 0.1666666679084301), ('oldpeak', False, -0.82251), ('reversable_defect', False, 0.375)]
0.9041095890410958 0.2783832093640085 0.5186470084430951 0.7036097560975612
merge complete for batch_idx 14 (39) patterns
start get explainer for batch_idx 14
Working on CHIRPS for instance 15 of 27
as_chirps for batch_idx 15
start mining for batch_idx 15 with support = 0.1
found 37 patterns from 500 trees for batch_idx 15
start score sort for batch_idx 15 (37) patterns
start merge rule for batch_idx 15 (37) patterns
[('chest_pain', True, 0.66667), ('reversable_defect', True, -0.25)]
0.967032967032967 0.3582416706520007 0.13637248952076117 0.22859312701727735
merge complete for batch_idx 15 (37) patterns
start get explainer for batch_idx 15
as_chirps for batch_idx 16
start mining for batch_idx 16 with support = 0.1
found 60 patterns from 320 trees for batch_idx 16
start score sort for batch_idx 16 (60) patterns
start merge rule for batch_idx 16 (60) patterns
[('number_of_major

In [9]:
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
k = 0
for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/test_size[k])
        rule_length[name].append(i[2][name]/test_size[k])
        if len(i[3][name]) >= 1:
            f_precision[name].append(np.array(i[3][name]).mean())
        f_time[name].append(np.array(i[4][name]).mean())
    k = + 1
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',
          np.array(rule_length[name]).mean(),',',
          np.array(f_precision[name]).mean(),',',
          np.array(f_time[name]).mean())

gs 0.21371742112482855 , 3.3407407407407406 , 0.9660612648221344 , 0.0011778319323504413
ls 0.19368998628257889 , 3.5703703703703704 , 0.9303317364427613 , 2.1616683871657765
an 0.2172839506172839 , 2.4703703703703703 , 0.6053572456573664 , 10.687307157339873
lf 0.042798353909465 , 7.959259259259258 , 1.0 , 1.1619171804851958
df 1.0 , 0.1 , 0.8925925925925924 , 0.1495398106398406
ch 0.3358024691358025 , 2.359259259259259 , 0.9812256056700501 , 0.3303267761513039
